In [35]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

In [36]:
data_df = pd.read_csv('/Users/ksenia/Analytics/Instacart/departments.csv')

In [37]:
data_df[:5]

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [38]:
data_df.dtypes

department_id     int64
department       object
dtype: object

In [39]:
def get_conn():
    return mysql.connector.connect(host= "localhost",
                  user="root",
                  passwd="password",
                  db="instacart")

In [40]:
# Create table Departments in MySQL
TABLES = {}
TABLES['departments'] = (
    "CREATE TABLE `departments` ("
    "  `department_id` int NOT NULL,"
    "  `department` varchar(200),"
    "  PRIMARY KEY (`department_id`), UNIQUE KEY `id` (`department_id`)"
    ") ENGINE=InnoDB")

conn = get_conn()
cursor = conn.cursor()
for name, ddl in TABLES.iteritems():
    try:
        print("Creating table {}: ", name)
        cursor.execute(ddl)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
conn.commit()
cursor.close()
conn.close()

('Creating table {}: ', 'departments')
OK


In [41]:
TABLES

{'departments': 'CREATE TABLE `departments` (  `department_id` int NOT NULL,  `department` varchar(200),  PRIMARY KEY (`department_id`), UNIQUE KEY `id` (`department_id`)) ENGINE=InnoDB'}

In [42]:
#List of headers for inserting data into database
headers = 'department_id, department'
headers = headers.split(", ")
print(headers)

['department_id', 'department']


In [43]:
#Function to insert each line with data into database under corresponding headers 
def create_insert_sql(line, headers):
    """
    Create a SQL statement for inserting row 
    """
    values = []
    for header in headers:
        val = line[header]
        
        if repr(val) == 'nan':
            values.append('null')
        else:
            values.append(repr(val))

    headerpart = ', '.join(headers)    
    valuespart = ', '.join(values)

    sql = 'INSERT INTO instacart.departments (' + headerpart + ')' + ' VALUES' + '(' + valuespart + ')'
    return sql


#Test funtion
line = data_df.iloc[2,:]
print(create_insert_sql(line, headers))

INSERT INTO instacart.departments (department_id, department) VALUES(3, 'bakery')


In [45]:
data_df

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [46]:
#Write product data into database

conn = get_conn()
cursor = conn.cursor()

try:
    for row in range(0,data_df.shape[0]):
        if row % 1000 == 0: 
            print(row)
            conn.commit()
        line = data_df.iloc[row,0:]
        print(line)
        sql = create_insert_sql(line, headers)
        try:
            cursor.execute(sql)
        except Exception as e:
            print(sql)
            raise e
    print("all done")

    
finally:
    cursor.close()
    conn.close()

0
department_id         1
department       frozen
Name: 0, dtype: object
department_id        2
department       other
Name: 1, dtype: object
department_id         3
department       bakery
Name: 2, dtype: object
department_id          4
department       produce
Name: 3, dtype: object
department_id          5
department       alcohol
Name: 4, dtype: object
department_id                6
department       international
Name: 5, dtype: object
department_id            7
department       beverages
Name: 6, dtype: object
department_id       8
department       pets
Name: 7, dtype: object
department_id                  9
department       dry goods pasta
Name: 8, dtype: object
department_id      10
department       bulk
Name: 9, dtype: object
department_id               11
department       personal care
Name: 10, dtype: object
department_id              12
department       meat seafood
Name: 11, dtype: object
department_id        13
department       pantry
Name: 12, dtype: object
department_id 